In [ ]:
#https://docs.microsoft.com/en-us/azure/machine-learning/tutorial-auto-train-models

In [ ]:
import logging

automl_settings = {
    "iteration_timeout_minutes": 2,
    "experiment_timeout_hours": 0.3,
    "enable_early_stopping": True,
    "primary_metric": 'spearman_correlation',
    "featurization": 'auto',
    "verbosity": logging.INFO,
    "n_cross_validations": 5
}

In [ ]:
subscription_id = "80b2bae0-9f97-4190-a0d1-f57c078442d8"
resource_group = "automl"
workspace_name = "autolmwa"
workspace_region = "West US"

# Import the Workspace class and check the Azure ML SDK version.
from azureml.core import Workspace

#ws = Workspace.create(name = workspace_name,
#                      subscription_id = subscription_id,
#                      resource_group = resource_group, 
#                      location = workspace_region)
#ws.get_details()

ws = Workspace(workspace_name = workspace_name,
               subscription_id = subscription_id,
               resource_group = resource_group)

# Persist the subscription id, resource group name, and workspace name in aml_config/config.json.
ws.write_config()

In [ ]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(task='regression',
                             debug_log='automated_ml_errors.log',
                             training_data=x_train,
                             label_column_name="totalAmount",
                             **automl_settings)

In [ ]:
from azureml.core.experiment import Experiment
experiment = Experiment(ws, "taxi-experiment")
local_run = experiment.submit(automl_config, show_output=True)

In [ ]:
from azureml.widgets import RunDetails
RunDetails(local_run).show()

In [ ]:
best_run, fitted_model = local_run.get_output()
print(best_run)
print(fitted_model)

In [ ]:
y_test = x_test.pop("totalAmount")

y_predict = fitted_model.predict(x_test)
print(y_predict[:10])

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

y_actual = y_test.values.flatten().tolist()
rmse = sqrt(mean_squared_error(y_actual, y_predict))
rmse

In [ ]:
sum_actuals = sum_errors = 0

for actual_val, predict_val in zip(y_actual, y_predict):
    abs_error = actual_val - predict_val
    if abs_error < 0:
        abs_error = abs_error * -1

    sum_errors = sum_errors + abs_error
    sum_actuals = sum_actuals + actual_val

mean_abs_percent_error = sum_errors / sum_actuals
print("Model MAPE:")
print(mean_abs_percent_error)
print()
print("Model Accuracy:")
print(1 - mean_abs_percent_error)